In [1]:
#INSTALATION DES DEPENDANCES

from datetime import datetime
from packaging import version
import tensorflow as tf

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

tf.keras.backend.set_floatx('float64')
# Load the TensorBoard notebook extension.
%load_ext tensorboard

TensorFlow version:  2.0.0-alpha0
The tensorboard module is not an IPython extension.


In [2]:
# DEFINIR LE MODELE

# le reseau de neurones va tenté de predire les X et les Y en sortie
class Model(tf.keras.Model):
    def __init__(self, x, y):
      super(Model, self).__init__()

# CREATION DES INPUTS

      # création de X, c'est une constante de type "float"
      self.x_input = tf.constant(x, dtype=tf.float64)
      # création de Y, c'est une constante de type "float64"
      self.y_input = tf.constant(y, dtype=tf.float64)

#CREATION DE LA 1ere COUCHE 2x2 (layer1)

      # Création du poids d'entrée W1, c est une variable, on prends 2 neuronnes en entrée et créer 2 neurones, la valeur minimale et 0 et la valeur max                est 1. type float. le trainable permet d'ajouter de la descente de gradient et son nom est W1
      self.W1 = tf.Variable(tf.random.uniform([2,2], minval=0., maxval=1., dtype=tf.float64), trainable = True, name='W1')

      # Création du poids d'entrée b1, c est une variable, on prends 2 neuronnes en entrée
      self.b1 = tf.Variable(tf.random.uniform([2], minval=0., maxval=1., dtype=tf.float64), trainable = True, name='b1')


#CREATION DE LA 2eme COUCHE 2x1 (layer2)

    # Création du poids d'entrée W2, c est une variable, on prends 2 neuronnes en entrée et créer 1 neurones,
      self.W2 = tf.Variable(tf.random.uniform([2,1], minval=0., maxval=1., dtype=tf.float64), trainable = True, name='W2')
      
    # Création du poids d'entrée B2, c est une variable, on prends 1neuronnes en entrée
      self.b2 = tf.Variable(tf.random.uniform([1], minval=0., maxval=1., dtype=tf.float64), trainable = True, name='b2')


# CREATION DE FONCTIONS DE CALCUL
# appel du reseau de neuronnes


    # definition d'un input constant de type float 
    def call(self, inputs):
      inputs = tf.constant(inputs, dtype=tf.float64)
   # tf.linalg.matmul est une multiplication 
   # tf.add est une adition 
      in_neurons_hidden_layer = tf.add(tf.linalg.matmul(inputs, self.W1),self.b1)  # J1= x*W1+b1
    # calcul de l output 
    # tf.sigmoid est le calcul de la sigmoid 
      out_neurons_hidden_layer = tf.sigmoid(in_neurons_hidden_layer)

    # calcul de l output layer

      in_neurons_output_layer = tf.add(tf.linalg.matmul(out_neurons_hidden_layer, self.W2),self.b2) # J2= x*W2+b2

      out_neurons_output_layer = tf.sigmoid(in_neurons_output_layer)

      return out_neurons_output_layer

In [3]:
# apprentissage du modele

import numpy as np
import pandas as pd


#XOR pour expliquer un reseau de neuronnes 
x = np.array([[0.,0.],[1.,1.],[0.,1.],[1.,0.]])
y = np.array([0.,0.,1.,1.])

pd.DataFrame({
    'input1':np.vstack(x).T[0],
    'input2': np.vstack(x).T[1],
    'output':y
})

,input1,input2,output
0,0.0,0.0,0.0
1,1.0,1.0,0.0
2,0.0,1.0,1.0
3,1.0,0.0,1.0


In [12]:


import sys

# Clear any logs from previous runs
#!rm -rf ./logs/ 
#Create logs
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/gradient_tape/' + current_time + '/train'

# creer un fichier pour Tensorboard pour monitorer les neuronnes 
summary_writer = tf.summary.create_file_writer(logdir)

model = Model(x, y)
optimizer = tf.optimizers.SGD(learning_rate=0.9)


# DEFINITION DE LA DESCENTE DE GRADIENT


# cette fonction est la difference entre le resultat et ce que lon attends 
def loss(outputs_model, targets):
  # soustraction de la valeure de l'on attends par rapport au modele
  error = tf.math.subtract(targets,outputs_model)
  return tf.reduce_sum(tf.square(error))


# cette fonction est la descente de gradiant 
def get_gradient(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model(inputs), targets)
  return tape.gradient(loss_value, [model.W1, model.b1, model.W2, model.b2])



# cette fonction est la boucle du modele
def run_network(inputs, targets, epochs):
  for i in range(epochs):
    grads=get_gradient(model, inputs, targets)

# nous allons envoyé une liste ZIP de nos gradiants
    optimizer.apply_gradients(zip(grads, [model.W1, model.b1, model.W2, model.b2]))
    # calcul des erreurs pour les afficher 
    loss_epoch = loss(model(inputs), model.y_input)
    with summary_writer.as_default():
      # afficher l'erreur
      tf.summary.scalar('loss', loss_epoch, step=i)

    if i % 100 == 0 :
      print("loss at the epoch ", i , loss_epoch) 
       
      
      

# le run_network compile tout 
# epochs est le nombre de fois que l on fait la boucle du modele
run_network(x,y,epochs=200)

loss at the epoch  0 tf.Tensor(5.5435032760416645, shape=(), dtype=float64)
loss at the epoch  100 tf.Tensor(4.0022872912374545, shape=(), dtype=float64)
